In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import mayfly as mf
import scipy.signal
import scipy.stats
import scipy.interpolate

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'datasets/data')
#SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""



In [ ]:
#os.mkdir(os.path.join(DATAPATH, 'dl', 'pca'))

In [ ]:
os.listdir(os.path.join(DATAPATH, 'dl'))

In [ ]:
os.listdir(os.path.join(DATAPATH))

# select templates/signals

In [ ]:
h5template = h5py.File(os.path.join(DATAPATH, '211027_84_25_2cm.h5'))
h5signal = h5py.File(os.path.join(DATAPATH, '211105_84_25_2cm_random.h5'))


In [ ]:
h5template['data'].shape
h5signal['data'].shape

In [ ]:
1474560 / (8192 * 60)

# Reduce templates/signals to 8192 samples, convert to tensor

In [ ]:
signal_array_cmplx = h5signal['data'][:, :].reshape((h5signal['data'].shape[0], 60, h5signal['data'].shape[-1] // 60))[:, :, 0:8192]
template_array_cmplx = h5template['data'][:, :].reshape((h5template['data'].shape[0], 60, h5template['data'].shape[-1] // 60))[:, :, 0:8192]

In [ ]:
print(signal_array_cmplx.shape)
print(template_array_cmplx.shape)

In [ ]:
signal_array = np.zeros((494, 2, 60, 8192))
template_array = np.zeros((12221, 2, 60, 8192))

signal_array[:, 0, :, :] = signal_array_cmplx.real
signal_array[:, 1, :, :] = signal_array_cmplx.imag

template_array[:, 0, :, :] = template_array_cmplx.real
template_array[:, 1, :, :] = template_array_cmplx.imag

signal_tensor = torch.as_tensor(signal_array)
template_tensor = torch.as_tensor(template_array)

print(signal_tensor.shape)
print(template_tensor.shape)

In [ ]:
custom_conv = torch.nn.Conv2d(2, 12221, (60, 8192), padding_mode='circular', bias='False')

In [ ]:
custom_conv.weight = torch.nn.Parameter(data = template_tensor, requires_grad=False)

In [ ]:
plt.plot(template_tensor[11000, 0, 0, :])
plt.plot(custom_conv.weight[11000, 0, 0, :])
plt.xlim(0, 100)

# Convert to tensors

In [ ]:
#signal_tensor = torch.zeros((494, 2, 60, 8192))
#template_tensor = torch.zeros(12221, 2, 60, 8192)


signal_tensor[:, 0, :, :] = signal_array.real
signal_tensor[:, 1, :, :] = signal_array.imag
template_tensor[:, 0, :, :] = template_array.real
template_tensor[:, 1, :, :] = template_array.imag

# load data

In [ ]:
# signal data
data = h5py.File(os.path.join(DATAPATH, 'dl',  '211203_dl_classification_84_25_2cm_slice1_sample2x8192_no_sum.h5'), 'r')



In [ ]:
data['train']['data'].shape

In [ ]:
real = data['train']['data'][np.arange(0, 12221, 2), 0, :]
imag = data['train']['data'][np.arange(0, 12221, 2), 1, :]
cmplx = real + 1j * imag

In [ ]:
noise = np.random.normal(loc=0, scale = np.sqrt(1.38e-23 * 10 * 50 * 1 * 200e6 / (2 * 2 * 8192)), size=2*8192)

In [ ]:
plt.plot(noise)
plt.plot(real[6110, 0, :])
plt.plot(imag[6110, 0, :])

In [ ]:
avg_spectrum = abs(cmplx).mean(axis=(0, 1))

In [ ]:
plt.plot(np.sort(avg_spectrum))

down_select_inds = np.argwhere(avg_spectrum > 0.5e-11)[:, 0]
print(down_select_inds.shape)

In [ ]:
len(np.arange(0, 12221, 2))

In [ ]:
name = '211210_dl_classification_84_25_2cm_slice1_sample2x8192_downselect6111x2x60x8335.h5'
data = h5py.File(os.path.join(DATAPATH, 'dl',  '211203_dl_classification_84_25_2cm_slice1_sample2x8192_no_sum.h5'), 'r')
signal_inds = np.arange(0, 12221, 2)


DownselectData(data, signal_inds, down_select_inds, os.path.join(DATAPATH, 'dl', name))

# load eigenvectors

In [ ]:
os.listdir(os.path.join(PATH, 'results', 'damselfly', 'pca'))

In [ ]:
pca_result = np.load(os.path.join(PATH, 'results', 'damselfly', 'pca', '211208_84_25_2cm_pca.npz'))

evecs = pca_result['evecs'].T
evals = pca_result['evals']

In [ ]:
plt.plot(evecs[256, :])

In [ ]:
name = '211209_dl_classification_84_25_2cm_slice1_sample2x8192_proj256x128.h5'
n_evecs = 256
n_project = 128

ProjectDataset2D(data, evecs, n_evecs, n_project, os.path.join(DATAPATH, 'dl', 'pca', name))



In [ ]:
os.listdir(os.path.join(DATAPATH, 'dl', 'pca'))

In [ ]:
# signal data
data = h5py.File(os.path.join(DATAPATH, 'dl','pca', '211209_dl_classification_84_25_2cm_slice1_sample2x8192_proj256x128.h5'), 'r')

In [ ]:
noise = np.random.normal(loc=0, scale = np.sqrt(1.38e-23 * 10 * 50 * 60 * 200e6 / (2 * 2 * 8192)), size=256 * 128)
noise = noise.reshape((256, 128))

In [ ]:
example = data['train']['data'][13, 0, :, :]

plt.plot(noise.flatten())
plt.plot(example.flatten())

#plt.imshow(abs(example + noise), aspect='auto', interpolation='none')
#plt.colorbar()
#proj_data = data[:, :, projection_inds].reshape((12221, 2, 256, 128))

In [ ]:
example = data['train']['data'][5, 0, :, :]
plt.imshow(abs(example + noise), aspect='auto', interpolation='none')
plt.colorbar()

In [ ]:
plt.plot(abs(evecs[3000, np.flip(np.argsort(abs(evecs[3000, :])))]))

# define output dataset parameters

In [ ]:
nsignal = data.shape[0]
nsample = data.shape[1]

# same signals in train, test, val sets. Different noise samples added to signals at run time
#ncopies_train = 10
#ncopies_test = 4

nslice = 1
ninput_ch = 2 # real, imag
slicesize = 2 * 8192

train_shape = (nsignal, nslice * ninput_ch, slicesize)
test_shape = (nsignal, nslice * ninput_ch, slicesize)

noise_temp = 10
fsample = 200e6
system_z = 50
nch = 60
kB = 1.38e-23

noise_var = kB * nch * noise_temp * system_z * fsample
noise_var_per_bin = noise_var / slicesize

name = os.path.join(DATAPATH, 'dl', '211203_dl_classification_84_25_2cm_slice1_sample2x8192.h5')
label = 'class'

config = {
    'train_shape': train_shape,
    'test_shape': test_shape,
    'nsignal': nsignal,
    'nsample': nsample,
    'nslice': nslice,
    'ninput_ch': ninput_ch,
    'slicesize': slicesize,
    'noise_temp': noise_temp,
    'fsample': fsample,
    'system_z': system_z,
    'nch': nch,
    'noise_var': noise_var,
    'noise_var_per_bin': noise_var_per_bin,
    'name': name,
    'label': label
}


CreateDLDataset(config, data, metadata, kass_data, kass_metadata)


